## ElasticNet Regresyon Modeli

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
ms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
y = df["Salary"] 
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis= 1).astype("float64")
X = pd.concat([X_, ms[["League_N", "Division_W", "NewLeague_N"]]], axis= 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 42)

In [3]:
from sklearn.linear_model import ElasticNet

In [4]:
enet_model = ElasticNet().fit(X_train, y_train)

D:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8487984.799367279, tolerance: 3898.686956380658
  positive)


In [5]:
enet_model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

In [6]:
enet_model.intercept_

-6.465955602113127

### TAHMİN

In [8]:
y_pred = enet_model.predict(X_test)

In [9]:
np.sqrt(mean_squared_error(y_test, y_pred))

357.1676548181244

In [10]:
r2_score(y_test, y_pred)

0.41070222469326945

### Model Tuning

In [11]:
from sklearn.linear_model import ElasticNetCV

In [13]:
enet_cv_model = ElasticNetCV(cv= 10, random_state= 0).fit(X_train, y_train)

In [14]:
enet_cv_model.alpha_

5230.7647364798695

In [15]:
enet_tuned = ElasticNet(alpha= enet_cv_model.alpha_).fit(X_train, y_train)

In [18]:
y_pred = enet_tuned.predict(X_test)

394.15280563218795

In [19]:
np.sqrt(mean_squared_error(y_test, y_pred))

394.15280563218795

### ÖNEMLİ NOTLAR!!

#### - "alphas" parametresi için bir liste oluşturmadık ve bunu modele bıraktık. ElasticNetCV fonksiyonu uygun olan değeri buldu. Eğer oluşturmak istenirse alphas = [.,.,.,.] şeklinde bir liste oluşturulabilir ve ElasticNetCV içerisine argüman olarak ifade edilebilir.
#### - "l1_ratio" parametresi: Bu parametre değeri 0 olduğunda L2 cezalandırması, 1 olduğunda ise L1 cezalandırması yapar. Dolayısıyla 0-1 arasında değişimi aslında cezalandırma metotlarının göreceli etkilerini ifade eder. Uygulamamızda ikisinin de etkisinin aynı düzeyde olmasını istediğimizden dolayı bunun ön tanımlı değerini olduğu gibi bıraktık: 0.5 .
#### - "l1_ratio" ve "alphas" beraber düşünülürse "l1_ratio" için aranacak bir liste verildiğinde yani ceza türlerinin etkilerini ayarlamak için bir liste verildiğinde her bir "l1_ratio" değeri için farklı "alphas" değeri olacaktır ki bu durumda bu alphalar ceza terimleri etkilerini modele yansıtmış olacak.